In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
model_path = "/gdrive/MyDrive/PPNCKH/saved/SST2/BERT"

# Data downloader


In [3]:
!pip install tokenizers -q
!pip install transformers -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.4 MB/s eta 0:00:00


In [4]:
# Random
from transformers import set_seed
set_seed(42)

In [5]:
from datasets import load_dataset
from datasets import DatasetDict

train_dataset = load_dataset("sst2", split = "train")
test_dataset = load_dataset("sst2", split = "validation")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


Split the train into train and dev

In [6]:
train, dev = train_dataset.train_test_split(
    test_size = .3,
    seed = 42,
    stratify_by_column = "label"
).values()

In [7]:
train

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 47144
})

In [8]:
dev

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 20205
})

In [9]:
test_dataset

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 872
})

In [10]:
dataset = DatasetDict({
    "train" : train,
    "dev" : dev,
    "test" : test_dataset
})

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 47144
    })
    dev: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 20205
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
})

In [12]:
num_labels = len(dataset["train"].features["label"].names)

In [13]:
num_labels

2

# Preprocessing

In [14]:
from transformers import AutoTokenizer

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast = True)

In [16]:
tokenizer(dataset["train"][0]["sentence"])

{'input_ids': [101, 1893, 1183, 1105, 4888, 1183, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation = True)

In [18]:
tokenized_dataset = dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/47144 [00:00<?, ? examples/s]

Map:   0%|          | 0/20205 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Rename the label column to fit the concept

In [19]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [20]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 47144
    })
    dev: Dataset({
        features: ['idx', 'sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20205
    })
    test: Dataset({
        features: ['idx', 'sentence', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
})

# Finetuning BERT for text classification

In [21]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [22]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = num_labels)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [23]:
batch_size = 16
learning_rate = 2e-5
weight_decay = .01
num_train_epochs = 10
eval_steps = 500
logging_steps = len(tokenized_dataset["train"]) // batch_size
early_stopping_patience = 3

In [24]:
args = TrainingArguments(
    "/saved",
    evaluation_strategy = IntervalStrategy.STEPS,
    # save_strategy = IntervalStrategy.STEPS,

    eval_steps = eval_steps,
    logging_steps = logging_steps,
    
    learning_rate = learning_rate,
    
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    
    num_train_epochs = num_train_epochs,
    
    weight_decay = weight_decay,
    save_total_limit = 3,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    # report_to = "wandb"
)

In [25]:
from datasets import load_metric
import numpy as np

In [26]:
metric = load_metric("glue", "mnli")

<ipython-input-26-1c23e2ff899a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mnli")


In [27]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis = 1)

    return metric.compute(predictions = predictions, references = labels)

In [28]:
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["dev"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

## Training

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,No log,0.241553,0.904083
1000,No log,0.225581,0.918387
1500,No log,0.204509,0.924920
2000,No log,0.187754,0.934670
2500,No log,0.179335,0.937293
3000,0.254800,0.216795,0.940906


KeyboardInterrupt: ignored

In [ ]:
# Saving pt file
trainer.save_model(model_path)

# Evaluation

In [29]:
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = num_labels)

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["dev"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Count parameters

In [31]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [32]:
count_parameters(model)

+---------------------------------------------------------+------------+
|                         Modules                         | Parameters |
+---------------------------------------------------------+------------+
|          bert.embeddings.word_embeddings.weight         |  22268928  |
|        bert.embeddings.position_embeddings.weight       |   393216   |
|       bert.embeddings.token_type_embeddings.weight      |    1536    |
|             bert.embeddings.LayerNorm.weight            |    768     |
|              bert.embeddings.LayerNorm.bias             |    768     |
|     bert.encoder.layer.0.attention.self.query.weight    |   589824   |
|      bert.encoder.layer.0.attention.self.query.bias     |    768     |
|      bert.encoder.layer.0.attention.self.key.weight     |   589824   |
|       bert.encoder.layer.0.attention.self.key.bias      |    768     |
|     bert.encoder.layer.0.attention.self.value.weight    |   589824   |
|      bert.encoder.layer.0.attention.self.value.bi

108311810

In [ ]:
trainer.evaluate()

{'eval_loss': 0.21679489314556122,
 'eval_accuracy': 0.94090571640683,
 'eval_runtime': 47.346,
 'eval_samples_per_second': 426.752,
 'eval_steps_per_second': 26.676}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
print(classification_report(predictions, labels))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90       392
           1       0.95      0.88      0.91       480

    accuracy                           0.91       872
   macro avg       0.91      0.91      0.91       872
weighted avg       0.91      0.91      0.91       872



In [ ]:
print(f"Accuracy = {accuracy_score(predictions, labels):.6f}, \
        Precision = {precision_score(predictions, labels, average = 'macro'):.6f}, \
        Recall = {recall_score(predictions, labels, average = 'macro'):.6f}, \
        F1 = {f1_score(predictions, labels, average = 'macro'):.6f}")

Accuracy = 0.905963,         Precision = 0.905174,         Recall = 0.909205,         F1 = 0.905628


In [ ]:
print(confusion_matrix(predictions, labels))

[[369  23]
 [ 59 421]]


In [ ]:
def print_clarification(idx):
    for sent, pred, label in zip(tokenized_dataset["test"][0:idx]["sentence"], predictions[0:idx], labels[0:idx]):
        print(sent)
        print(f"Predicted: {pred} - Label: {label}")

In [ ]:
print_clarification(10)

it 's a charming and often affecting journey . 
Predicted: 1 - Label: 1
unflinchingly bleak and desperate 
Predicted: 0 - Label: 0
allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
Predicted: 1 - Label: 1
the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
Predicted: 1 - Label: 1
it 's slow -- very , very slow . 
Predicted: 0 - Label: 0
although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . 
Predicted: 1 - Label: 1
a sometimes tedious film . 
Predicted: 0 - Label: 0
or doing last year 's taxes with your ex-wife . 
Predicted: 0 - Label: 0
you do n't have to know about music to appreciate the film 's easygoing blend of comedy and romance . 
Predicted: 1 - Label: 1
in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 sank from quirky to jerky to utter tur

In [ ]:
for i in range(len(predictions)):
    if predictions[i] != labels[i]:
        print(tokenized_dataset["test"][i]["sentence"])
        print(f"Predicted: {predictions[i]} - Label: {labels[i]}")

holden caulfield did it better . 
Predicted: 1 - Label: 0
it offers little beyond the momentary joys of pretty and weightless intellectual entertainment . 
Predicted: 1 - Label: 0
the script kicks in , and mr. hartley 's distended pace and foot-dragging rhythms follow . 
Predicted: 1 - Label: 0
this one is definitely one to skip , even for horror movie fanatics . 
Predicted: 1 - Label: 0
you wo n't like roger , but you will quickly recognize him . 
Predicted: 1 - Label: 0
if steven soderbergh 's ` solaris ' is a failure it is a glorious failure . 
Predicted: 0 - Label: 1
this riveting world war ii moral suspense story deals with the shadow side of american culture : racial prejudice in its ugly and diverse forms . 
Predicted: 1 - Label: 0
it 's one of those baseball pictures where the hero is stoic , the wife is patient , the kids are as cute as all get-out and the odds against success are long enough to intimidate , but short enough to make a dream seem possible . 
Predicted: 0 - Labe

Inference time measuring

In [ ]:
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def predict(text, model):
    tokenized = tokenizer(text, return_tensors = "pt").to(device)
    output = model(**tokenized)[0].detach().cpu().numpy()
    
    prediction = np.argmax(output, axis = 1)[0]

    return prediction

In [ ]:
predict("lathan and diggs have considerable personal charm , and their screen rapport makes the old story seem new", model)

1

In [ ]:
from time import perf_counter

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
predicted_timelapse = []

for text in tqdm_notebook(tokenized_dataset["test"]["sentence"]):
    start_time = perf_counter()
    predict(text = text, model = model)
    end_time = perf_counter()

    predicted_timelapse.append(end_time - start_time)

<ipython-input-50-ac57539ee258>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm_notebook(tokenized_dataset["test"]["sentence"]):


  0%|          | 0/872 [00:00<?, ?it/s]

In [ ]:
timelapse_mean = np.mean(predicted_timelapse) * 1000
timelapse_std = np.std(predicted_timelapse) * 1000

In [ ]:
print(f"Mean: {timelapse_mean:.8f}, stdev: {timelapse_std:.8f} ms")

Mean: 16.98353610, stdev: 4.22764945 ms
